**arcasHLA typing**
---

From [arcasHLA](https://github.com/RabadanLab/arcasHLA/tree/master).

In [ ]:
from arcas_hla_runner import ArcasHLARunner
from pathlib import Path

In [ ]:
runner = ArcasHLARunner()
runner.set_arcas_hla_path("/home/jovyan/arcasHLA/arcasHLA")

In [ ]:
main_dir = '/home/jovyan/researcher_home/isaak_RNAseq/GC130445'
main_dir_path = Path(main_dir)

genes = "all" # A, B, C, DMA, DMB, DOA, DOB, DPA1, DPB1, DQA1, DQB1, DRA, DRB1, DRB3, DRB5, E, F, G, H, J, K, L


bam_file_path = main_dir_path / (main_dir_path.stem + '.bam')
extract_reads_dir_path = main_dir_path / 'extract_reads'
extract_reads_dir_path.mkdir(parents=True, exist_ok=True)

fq_file_path = extract_reads_dir_path / (bam_file_path.stem + '.extracted.fq.gz')
genotype_dir_path = main_dir_path / 'output_genotype'
genotype_dir_path.mkdir(parents=True, exist_ok=True)

genotype_file_path = genotype_dir_path / (bam_file_path.stem + '.genotype.json')
partial_dir_path = main_dir_path / 'output_partial'
partial_dir_path.mkdir(parents=True, exist_ok=True)


In [ ]:
# Step 1: Extract reads
runner.extract_reads(bam_file_path, extract_reads_dir_path, threads=8, verbose=True, single=True)

# FASTQ analysis script

The Python script below analyzes a FASTQ file to provide key statistics about the sequencing reads and their quality scores. Specifically, it performs the following tasks:

1. **Read Length Analysis**:
   - Counts the total number of reads.
   - Calculates the average read length and its standard deviation.

2. **Phred Quality Score Analysis**:
   - Extracts Phred quality scores for all bases across all reads.
   - Computes the overall average quality score and its standard deviation.

In [ ]:
from Bio import SeqIO
import numpy as np
import gzip

# Initialize variables
read_lengths = []
quality_scores = []

# Parse the compressed FASTQ file
with gzip.open(fq_file_path, "rt") as handle:
    for record in SeqIO.parse(handle, "fastq"):
        read_lengths.append(len(record.seq))
        quality_scores.extend(record.letter_annotations["phred_quality"])
        
# Calculate read length statistics
num_reads = len(read_lengths)
average_length = np.mean(read_lengths)
std_dev_length = np.std(read_lengths)

# Calculate quality score statistics
average_quality = np.mean(quality_scores)
std_dev_quality = np.std(quality_scores)

# Print results
print(f"Number of reads: {num_reads}")
print(f"Average read length: {average_length:.2f}")
print(f"Standard deviation of read lengths: {std_dev_length:.2f}")
print(f"Average Phred quality score: {average_quality:.2f}")
print(f"Standard deviation of quality scores: {std_dev_quality:.2f}")

# Step 2: Genotype

In [ ]:
runner.genotype(
                fq_file_path,
                genotype_dir_path,
                genes=genes,
                # population="prior",
                min_count=1,
                tolerance=1e-7,
                # max_iterations=500,
                drop_iterations=4,
                threads=8,
                verbose=True,
                single=True,
                avg_fragment_length=50,
                std_fragment_length=1
            )


In [ ]:
# Step 3: Partial Genotyping
runner.partial_genotype(fq_file_path, 
                        genotype_file_path,
                        partial_dir_path,
                        genes=genes,
                        threads=8,
                        verbose=True,
                        single=True
                    )